# GTFS Realtime Data Retrieval
This notebook retrieves GTFS realtime data from TransLink for analysis.

## 1. Setup

In [1]:
import os
import sys
import pandas as pd

sys.path.append(os.path.dirname(os.path.abspath('.')))
from src.data_connection import DatabaseConnector

db_connector = DatabaseConnector()

2025-12-29 18:50:55.687693: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Output directory
OUTPUT_DIR = 'data/base_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [3]:
# Vancouver local time period settings
start_date_local = '20251118'
end_date_local = '20251124'

## 2. GTFS Realtime Data Retrieval

In [4]:
base_df = db_connector.read_sql(f"""
select 
base.*,
COALESCE(alerts.has_active_alert, 0) as has_active_alert,
COALESCE(alerts.high_impact_alert_count, 0) as high_impact_alert_count,
COALESCE(alerts.alert_police_activity, 0) as alert_police_activity,
COALESCE(alerts.alert_construction, 0) as alert_construction,
COALESCE(alerts.alert_technical_problem, 0) as alert_technical_problem,
COALESCE(alerts.alert_effect_no_service, 0) as alert_effect_no_service,
COALESCE(alerts.alert_effect_detour, 0) as alert_effect_detour,
COALESCE(alerts.alert_severity_score, 0) as alert_severity_score
from 
gtfs_realtime.gtfs_rt_base_forML_v base
LEFT JOIN gtfs_realtime.gtfs_rt_alert_features_route_hour_v alerts
    ON alerts.route_id = base.route_id
    AND alerts.alert_hour = base.time_bucket
where base.start_date >= '{start_date_local}' and base.start_date <= '{end_date_local}';
""")

# Ensure start_date is string
if 'start_date' in base_df.columns:
    base_df['start_date'] = base_df['start_date'].astype(str)

base_df.to_parquet(f'{OUTPUT_DIR}/analyze_delay_base_{start_date_local}_{end_date_local}.parquet', index=False)

In [5]:
print('Data loaded. Number of rows:', len(base_df))
print('Date range:', base_df['start_date'].min(), 'to', base_df['start_date'].max())

Data loaded. Number of rows: 5617501
Date range: 20251118 to 20251124


In [6]:
base_df.groupby('route_id').size().sort_values(ascending=False).head(10)

route_id
23384    298429
6636     260421
6627     231478
16718    227754
6617     217723
6622     144219
6705     138628
6624     131228
6619     129945
37810    113230
dtype: int64